In [1]:
root_path = "/content/drive/MyDrive"
import numpy as np
import pandas as pd 
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import  cross_val_predict
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from xgboost import XGBRegressor
%matplotlib inline

In [2]:
from google.colab import drive
import tensorflow as tf
print(tf.test.gpu_device_name())
drive.mount('/content/drive')

/device:GPU:0
Mounted at /content/drive


In [3]:
# parameters
varAOD = 'nAODm'
varPM = 'PMc'
features = [varAOD, 'lat', 'long', 'Prob_bestm', 'd2m', 't2m', 'blh',
            'sp', 'lai_hv', 'ws10', 'wd10', 'uvb', 'RH', 'DOY']
name = 'train' + ".csv"
path = os.path.join(root_path, 'final_data', name)

dftrain = pd.read_csv(path)


Xtrain = dftrain[features]
ytrain = dftrain.loc[:,['PMc']]
print(Xtrain)
scaler = MinMaxScaler()
Xstrain = scaler.fit_transform(Xtrain)
ytrain = ytrain.to_numpy()

          nAODm        lat       long  ...            uvb        RH  DOY
0      0.167574  35.697773  51.243110  ...  609384.375337  0.626860  289
1      0.562968  35.717510  51.385909  ...  435596.859771  0.866310   10
2      0.202551  35.727080  51.431200  ...  903657.588627  0.622771  227
3      0.188717  35.670050  51.297350  ...  962451.589145  0.602828  172
4      0.068609  35.727080  51.431200  ...  992356.247693  0.647341  147
...         ...        ...        ...  ...            ...       ...  ...
15843  0.199543  35.739888  51.267886  ...  983999.376254  0.551776  180
15844  0.176790  35.717510  51.385909  ...  989760.528345  0.525596  168
15845  1.686084  35.727080  51.431200  ...  585029.296545  0.826983   46
15846  0.134852  35.778232  51.428623  ...  633781.540136  0.562845  285
15847  0.148110  35.702270  51.350940  ...  926355.561885  0.756086  103

[15848 rows x 14 columns]


XGboost

In [4]:
model = XGBRegressor(n_estimators= 2000, random_sate = 123)

In [5]:
name = 'test' + ".csv"
path = os.path.join(root_path, 'final_data', name)

dftest = pd.read_csv(path)
Xtest = dftest[features]
ytest = dftest.loc[:,[varPM]]
ytest = ytest.to_numpy()
ytest = np.ravel(ytest)
Xstest = scaler.fit_transform(Xtest)

model.fit(Xstrain, ytrain)

[06:26:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(n_estimators=2000, random_sate=123)

In [6]:
########## Train
print('------------------Train------------------')
dftrain["y_pred"] = model.predict(Xstrain)

def NominalPM(row):
    return row["y_pred"]/((1-row["RH"])**(-1))
dftrain["PMp"] = dftrain.apply(NominalPM, axis = 1)
RMSE = np.round(mean_squared_error(dftrain["PM2.5"], dftrain["PMp"], squared=False),2)
MAE = np.round(mean_absolute_error(dftrain["PM2.5"], dftrain["PMp"]),2)
print ("RMSE: ", RMSE)
print ("MAE: ", MAE)
from scipy.stats import pearsonr
corr, _ = pearsonr(dftrain["y_pred"], dftrain["PMc"])
print('Pearsons R2 correlation: %.2f' % corr**2)
APE = np.mean(abs(dftrain["PM2.5"]-dftrain["PMp"]))/np.mean(dftrain["PM2.5"])
print('APE: %.2f' % APE)

########## Test
print('------------------Test------------------')
dftest["y_pred"] = model.predict(Xstest)
dftest["PMp"] = dftest.apply(NominalPM, axis = 1)
RMSE = np.round(mean_squared_error(dftest["PM2.5"], dftest["PMp"], squared=False),2)
MAE = np.round(mean_absolute_error(dftest["PM2.5"], dftest["PMp"]),2)
print ("RMSE: ", RMSE)
print ("MAE: ", MAE)
corr, _ = pearsonr(dftest["y_pred"], dftest["PMc"])
print('Pearsons R2 correlation: %.2f' % corr**2)

APE = np.mean(abs(dftest["PM2.5"]-dftest["PMp"]))/np.mean(dftest["PM2.5"])
print('APE: %.2f' % APE)

------------------Train------------------
RMSE:  6.39
MAE:  4.79
Pearsons R2 correlation: 0.92
APE: 0.17
------------------Test------------------
RMSE:  8.97
MAE:  6.86
Pearsons R2 correlation: 0.74
APE: 0.24


In [ ]:
# Save model
output_path = os.path.join(root_path , "PythonCodes", "PMXGB")
model.save_model(output_path)
print(output_path)

/content/drive/MyDrive/PythonCodes/PMXGB
